In [13]:
import requests
import pandas as pd
from tqdm.notebook import tqdm

In [23]:
data = pd.read_csv("../data/old/yappy_hackaton_2024_400k.csv").iloc[40001:45000]
data.fillna("", inplace=True)

In [24]:
data.T.to_dict()

{40001: {'link': 'https://cdn-st.rutubelist.ru/media/0f/fc/00fc6050405ca29a97843d91b4ef/fhd.mp4',
  'description': 'Нравится ее стиль'},
 40002: {'link': 'https://cdn-st.rutubelist.ru/media/8a/04/c51a9e644f578ac328c778124d02/fhd.mp4',
  'description': 'Стесняшка Жэньши • Монолог фармацевта | Kusuriya no Hitorigoto'},
 40003: {'link': 'https://cdn-st.rutubelist.ru/media/9f/27/4edb34bc4ce182bfcd9c6822bd56/fhd.mp4',
  'description': '#спорт #наспорте #растяжка #фитнес #мотивация'},
 40004: {'link': 'https://cdn-st.rutubelist.ru/media/13/83/ed72c0e847c9964bd75061297e5a/fhd.mp4',
  'description': '#красивыедевушки #губы #макияж'},
 40005: {'link': 'https://cdn-st.rutubelist.ru/media/fb/73/16eb29534024a2cb4e9c99148cb0/fhd.mp4',
  'description': '#наспорте #тренировка #упражнения'},
 40006: {'link': 'https://cdn-st.rutubelist.ru/media/2f/bf/f97621424fd888f973477ce26c23/fhd.mp4',
  'description': '#киноновинки , #киноафиша , #кино  , #фильмы , #сериалы , #чтопосмотреть , #трейлер , #трейлеры'}

In [26]:
responses = []

headers = {
    'accept': 'application/json',
    'Content-Type': 'application/json',
}

for i in tqdm(range(500)):
    json_data = data.iloc[i].to_dict()
    response = requests.post('http://localhost:8080/index', headers=headers, json=json_data)
    if response.status_code != 200:
        print(i, response.text)
    responses.append(response)

  0%|          | 0/500 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [33]:
import aiohttp, asyncio

async def post_request(session, url, headers, json_data):
    async with session.post(url, headers=headers, json=json_data) as response:
        if response.status != 200:
            print(json_data, await response.text())
        return await response.json()

responses = []
batch_size = 5
url = 'http://localhost:8080/index'

async with aiohttp.ClientSession() as session:
    for i in tqdm(range(0, len(data), batch_size)):
        tasks = []
        for j in range(i, min(i + batch_size, len(data))):
            json_data = data.iloc[j].to_dict()
            tasks.append(post_request(session, url, headers, json_data))
        responses.extend(await asyncio.gather(*tasks))

  0%|          | 0/1000 [00:00<?, ?it/s]

CancelledError: 

In [22]:
type(json_data["description"])

float

In [32]:
len(tasks)

5